# **Segmentation – Basics**

In this notebook on segmentation, we will learn how to segment hematological images using different approaches. In a first step, we try to segment the cells using simple thresholding.

Note that several of the topics discussed in this notebook are also covered 
in this insightful tutorial for this ImageJ/Fiji plugin [MorphoLibJ](https://imagej.net/plugins/morpholibj)

---

## **Preparations**

Let's begin with the usual preparatory steps...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import PIL
from pathlib import Path

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Enable this line if you want to use the interactive widgets
# It requires the ipympl package to be installed.
#%matplotlib widget

import sys
sys.path.insert(0, "../")
import tools

Here, too, we will work with the same images as before in the notebook on pre-processing:

In [ ]:
# Read in the data
img1 = cv.imread("../data/images/hematology-baso1.jpg", cv.IMREAD_COLOR)
img2 = cv.imread("../data/images/hematology-baso2.jpg", cv.IMREAD_COLOR)
img3 = cv.imread("../data/images/hematology-blast1.jpg", cv.IMREAD_COLOR)

img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)
img3 = cv.cvtColor(img3, cv.COLOR_BGR2RGB)

tools.show_image_chain([img1, img2, img3], titles=["img1", "img2", "img3"])

---

## **Thresholding**

We can segment images using basic thresholding techniques. In this example, we explore the different thresholding methods available in OpenCV:
- Simple thresholding (use [`cv.threshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57), use flag `cv.THRESH_BINARY` or `cv.THRESH_BINARY_INV`)
- Adaptive thresholding (use [`cv.adaptiveThreshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3))
- Otsu's thresholding (use [`cv.threshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57), use flags `cv.THRESH_BINARY+cv.THRESH_OTSU`)

Thresholding segments pixels into either foreground or background based on their intensity values. Thresholding is therefore an instance of *binary* image segmentation. The algorithms work by comparing the intensity values of the pixels in an image with a threshold value. Pixels with intensity values greater than the threshold are classified as foreground pixels, while pixels with intensity values less than the threshold are classified as background pixels. The threshold value can be set manually or determined automatically.

As preparation, please read this documentation on segmentation methods in OpenCV: [Link](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)



In [ ]:
######################
###    EXCERISE    ###
######################

# Choose here the image to work with
img = img1

# 1) Summarize the three different thresholding techniques in own words. Which methods
#    use a global threshold, which ones apply a local threshold? 

# 2) Develop a strategy to segment the white blood cells (purple), the red blood cells 
# (red) and the background (white/gray). You may want to exploit the fact that we have
# colors to work with:
tools.show_image_chain([img[:,:,0], img[:,:,1], img[:,:,2]], titles=["R", "G", "B"])

# 3) Identify the different regions using thresholding
mask_wbc = ...
mask_rbc = ...
mask_bg = ...

# 4) Visualize the masks. Idea: combine the three masks into an RGB image.
mask_seg = ...

# 5) Discuss your results. What could be improved? What are the limitations of this 
#    approach? Are the masks mutually exclusive? Are they accurate?



In [ ]:
######################
###    SOLUTION    ###
######################

# 1) cv.threshold:              Apply a global threshold to the image.
#    cv.adaptiveThreshold:      Apply a local threshold to the image.
#    cv.threshold (otsu):       Apply a global threshold to the image using Otsu's method.

# 2) Segmentation strategy:
#    The information in the three different channels suggests that we can use the
#    red channel to segment the red blood cells, the blue channel to segment the white
#    blood cells and the luminance channel (or gray channel) to segment the background. 
#    We can then combine the three masks to obtain the final segmentation.
# 
#    Display the 3 channels. See note below as to why we disable normalization.
img = img1
tools.show_image_chain([img[:,:,0], img[:,:,1], img[:,:,2]], 
                       titles=["R", "G", "B"], normalize=False)

# 3) Let's try how this works in practice
def segment_blood_cells_thr(img, return_masks=False):

    # Smooth the image (to reduce noise)
    #img = cv.GaussianBlur(img, (5, 5), 0)

    # -> Convert image to grayscale
    gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

    # -> Extract background using Otsu's method
    thr_bg, mask_bg = cv.threshold(gray, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)

    # -> Extract red and white blood cells
    thr_rbc, mask_rbc = cv.threshold(img[:,:,0], 130, 255, cv.THRESH_BINARY)
    thr_wbc, mask_wbc = cv.threshold(img[:,:,2], 140, 255, cv.THRESH_BINARY)

    # -> Apply the segmentation logic: 
    #     - First observe that the background takes high values in the red and blue
    #       channels. Thresholding the red and blue channels will also include the 
    #       background. Furthermore, the blue component sometimes is also present 
    #       in the red blood cells. Therefore:
    #     - Exclude the background from the masks for the red and white blood cells
    #     - Exclude the red blood cells from the white blood cells mask. Here,
    #       we use the condition that something appears purple if the blue
    #       channel is significantly higher than the red channel.

    mask_rbc = mask_rbc.astype(bool) & ~mask_bg.astype(bool)
    mask_wbc = mask_wbc.astype(bool) & ~mask_bg.astype(bool)
    mask_wbc = mask_wbc & (img[:,:,0]*1.1 < img[:,:,2])

    # Combine the information into a color image.
    result = np.ones_like(img) * 255
    result[mask_rbc.astype(bool)] = [155, 107, 132]
    result[mask_wbc.astype(bool)] = [62, 32, 152]

    if return_masks:
        return result, mask_bg, mask_rbc, mask_wbc
    else:
        return result


# Compute the segmentation and viusalize the results
img = img1
ret = segment_blood_cells_thr(img, return_masks=True)
result1, mask_bg, mask_rbc, mask_wbc = ret

# Visualize the masks
tools.show_image_chain([mask_bg, mask_rbc, mask_wbc], 
                       titles=["Background", "RBC", "WBC"])
# Visualize the results
tools.show_image_chain([img, result1], titles=["Input: img1", "Output: Segmentation"]);


The segmentation looks fairly good, but it is not perfect. The main limitations are:
- The thresholding is very sensitive to the threshold values. Small changes in the threshold values can lead to very different results.
- The assumptions and the segmentation logic are very specific to the images at hand. They may not generalize well to other images.
- The masks contain holes and do not perfectly segment the cells. This is due to the thresholding method used.
- There are boundary effects visible (e.g. in the red blood cells), indicating that the thresholding strategy is not perfect.
- Two nearby cells may touch or overlap. In that case we cannot distinguish them in the segmentation. The result for `img3` shows this problem clearly, see below.

We can refine the results of this approach by...
- ...tuning the thresholding parameters
- ...by improving the segmentation logic
- ...by smoothing the image (see commented out line of code above)
- ...by using morphological operations (to close holes and remove noise in the segmentation masks, see next tutorial).

**Note**: When displaying the images, it is helpful to disable normalization, which changes the appearance of the image by stretching the pixel values to the full range of [0, 255]. For (single-channel) images, this is the default behavior of [`plt.imshow()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html), the underlying function we use in our convenience function `tools.show_image_chain()`. We want to keep the original values so that we can read the grayscale values from the image with a color picker. 

A color picker is a tool for measuring the current color value under the mouse pointer. On MacOS there is the [Digital Color Meter](https://support.apple.com/guide/digital-color-meter/welcome/mac) (installed by default under /System/Applications/Utilities/). On Windows, there is an equivalent tool *Color Picker* as part of the [PowerToys](https://learn.microsoft.com/en-us/windows/powertoys/). Ubuntu offers [Gpick](https://www.gpick.org/). These tools can display the color values in different formats, and you can use them to copy color values to the clipboard, etc.


In [ ]:
# Display the segmentation results also for the other images:
result2 = segment_blood_cells_thr(img2)
tools.show_image_chain([img2, result2], titles=["Input: img2", "Output: Segmentation"]);
result3 = segment_blood_cells_thr(img3)
tools.show_image_chain([img3, result3], titles=["Input: img3", "Output: Segmentation"]);

---


## **Color clustering**

Instead of segmenting the image, we could also try to classify the different regions. For this, we can use a clustering algorithm such as [K-means](https://en.wikipedia.org/wiki/K-means_clustering). The algorithm works by classifying pixels in an image into clusters based on their color similarity. The similarity is measured using the (Eucledian or non-Euclidean) distance between the pixel values. The algorithm iteratively assigns pixels to clusters based on their distance to the cluster centers and updates the cluster centers based on the mean of the pixels in the cluster. The process continues until the cluster centers converge. See here for a nice [visualization](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)) of the algorithm.

As a preparatory step, have a look at the following two tutorials:
- Machine Learning Master / Jason Brownlee on [color quantization](https://machinelearningmastery.com/k-means-clustering-in-opencv-and-application-for-color-quantization/)
- Shubhang Agrawal / Image segmentation using [k-means clustering](https://medium.com/swlh/image-segmentation-using-k-means-clustering-46a60488ae71) (the tutorial has a few flaws, please excuse). 



<!-- 
Resources:
# Nice way of depicting the bars
https://pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
# OpenCV
https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html
# Machine Learning Mastery
https://machinelearningmastery.com/k-means-clustering-in-opencv-and-application-for-color-quantization/
# Watershed
https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html
# Segmentation with Skimage 
https://github.com/ipython-books/cookbook-2nd-code/blob/master/chapter11_image/03_segmentation.ipynb
# Combination between thresholding and color clustering
https://towardsdatascience.com/image-color-segmentation-by-k-means-clustering-algorithm-5792e563f26e
-->

In [ ]:
######################
###    EXCERISE    ###
######################

# Choose here the image to work with
img = img1

# 1) Reshape the color pixels into a Mx3 matrix (M: number of pixels)
#    and convert the data type to float32.
data = img.reshape(-1, 3).astype(np.float32)

# 2) Apply the K-means algorithm to the data. Use the cv.kmeans function.
#    Choose the number of clusters K=3.
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 3
ret, label, centers = cv.kmeans(data, K, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
# label contains the cluster index for each pixel
# centers contains the cluster centers (colors!)

# 3) Reshape and convert the data back to uint8
img_seg = ...

# 4) Visualize the segmented image
tools.show_image_pair(img, img_seg, title1="Original", title2="Segmented");

# 5) Repeat the process for a different color space (e.g. HSV)
#    Is the clustering more robust? Why? When does this approach fail?


In [ ]:
######################
###    SOLUTION    ###
######################
def segment_blood_cells_kmeans(img, K=3, use_lab=False):
    # Blur the image to reduce noise (step is required here 
    # to yield feasible results)
    img = cv.GaussianBlur(img, (5, 5), 0)

    if use_lab:
        img = cv.cvtColor(img, cv.COLOR_RGB2LAB)

    # 1) Reshape the color pixels into a Mx3 matrix (M: number of pixels)
    #    and convert the data type to float32.
    data = img.reshape(-1, 3).astype(np.float32)
    
    # 2) Apply the K-means algorithm to the data. Use the cv.kmeans function.
    # Some parameters for the kmeans algorithm (termination criteria):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 0.1)
    ret, label, centers = cv.kmeans(data, 
                                    K=K, 
                                    bestLabels=None, 
                                    criteria=criteria, 
                                    attempts=10, 
                                    flags=cv.KMEANS_PP_CENTERS)
    # label contains the cluster index for each pixel
    # centers contains the cluster centers (colors!)

    # 3) Reshape and convert the data back to uint8
    img_seg = centers[label.flatten()].reshape(img.shape).astype(np.uint8)

    if use_lab:
        img_seg = cv.cvtColor(img_seg, cv.COLOR_LAB2RGB)

    # 4) Return the segmented image
    return img_seg


img = img1
img_seg = segment_blood_cells_kmeans(img, K=3, use_lab=False)
tools.show_image_chain([img, img_seg], titles=["Original", "Segmented"]);


The result here looks somewhat better than in the previous example, however,
the segmentation is still not perfect:
- It dependends on the prevalence of the colors in the image how well the clustering will work. If there are more nuances in the (e.g., background
  color) the clustering will yield more clusters representing the background, instead of identifying the blood cells.
- The K-means algorithm is sensitive to the initialization of the cluster centers. If the initialization is not done properly, the algorithm might   get stuck in a local minimum. 
- We still suffer in presence of touching and overlapping cells. While we can assign the right color to the cells, we cannot separate them.

We may be able to improve the results by using a different color space. For example, the LAB color space is more robust to changes in illumination and is more closely related to the human perception of color. Furthermore, we can allow more clusters to be found by the algorithm and then apply some post-processing to merge clusters that are similar. For example, if the cluster centers (= colors) have a dominant blue component, we can merge them into one cluster), etc.

---

## **Watershed algorithm for segmentation**

The watershed algorithm is a powerful tool for image segmentation. It is based on
the concept of watershed lines, which are lines that separate different regions in
an image. The algorithm works by flooding the image with "water" from different
regions (or seed points) and letting the water flow until it reaches a boundary. 
The watershed lines are then defined as the boundaries between the regions where
the water meets. The algorithm can be used to segment images into different regions
based on the intensity or color of the pixels. It is particularly useful for
segmenting images with complex shapes and structures.

Our dataset structurally resembles the image used in this [tutorial](https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html). 
Let's use this demo naively to segment our image using the watershed method. 

The tutorial uses the following strategy: 
1. Convert the image into binary mask using thresholding
2. Apply so-called morphological operations\* to remove noise, and to separate the objects. We also use similar operations to identify regions that most likely represent the background of our scene.
3. with the next steps we aim to identify the seed points for the watershed algorithm
   1. Apply the distance transform\*\* to the binary image, which measures the distance of each pixel to the nearest zero pixel.
   2. Apply a threshold to the distance-transformed image to obtain blobs of pixels that are located close to the centers of our objects of interest.
   3. Use the connected components\*\*\* algorithm to identify and enumerate the different seed points. This turns 
  the binary mask into a mask with (integer) labeled objects.
   4. In a last step, mark the background seed regions (see step 2.) with label 0.
4. Fiinally, apply the watershed algorithm to segment the regions of interest.
5. Visualize the segmented image.


### \* **Morphological operations** 
Morphological operations are a set of operations that are used to analyze and manipulate
the shape of objects in an image. Although the operations are defined also for other types
of images, they are most commonly used in binary images. The operations involve structuring
elements (kernels) that are used to probe the image and modify the pixel values based on the
interaction between the kernel and the image. The most common morphological operations are
dilation (expand the shapes), erosion (shrink the shapes), opening (dilation followed by
erosion), and closing (erosion followed by dilation). Morphological operations are used to 
remove noise, separate objects, and connect objects in an image. 

**Further reading**:
- OpenCV documentation. [Link](https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html)
- Beautiful illustration how morphological operations work: [Link](https://penny-xu.github.io/blog/mathematical-morphology)
- Wikipedia article on mathematical morphology. [Link](https://en.wikipedia.org/wiki/Mathematical_morphology)
- Blog post on morphological operations. [Link](https://towardsdatascience.com/7bcf1ed11756)

### \*\* **Distance transform**
The distance transform is useful for a variety of applications in image processing. It
permits to compute the distance of each pixel to the nearest boundary in a binary image.
Based on distance transforms, we can perform operations such as skeletonization, identify
geometric properties of objects or segment images. The algorithm works by iteratively 
propagating the distance values from the boundary pixels to the interior pixels. The 
distance is computed using a metric such as the Euclidean distance. It is relatively 
cheap to compute the distance transform.

**Further reading**
- Another application of the distance transform (in combination with watershed). [Link](https://docs.opencv.org/3.4/d2/dbd/tutorial_distance_transform.html)


### \*\*\* **Connected components**
Connected components in a binary image are regions of pixels that are touching each
other. In image processing, connected components are used to identify single objects 
in a segmentation mask. The connected components algorithm works by labeling each
pixel in the mask with a unique (integer) label based on the connectivity of the pixels. 

**Further reading**
- Wikipedia article on connected component labeling. [Link](https://en.wikipedia.org/wiki/Connected-component_labeling)

In [ ]:
######################
###    EXCERISE    ###
######################

img = img1

# Implement the approach lined out above. You can copy paste the code 
# from the above link and plug our image into it. Try to understand the
# code and the different steps. You may have to adjust the parameters
# to get a good segmentation result.

# https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html

...

In [ ]:
######################
###    SOLUTION    ###
######################
def segment_red_blood_cells_watershed(img):
    
    img = cv.GaussianBlur(img, (5, 5), 0)
    img_blur = cv.medianBlur(img, 5)
    
    #tools.show_image(img_blur)
    
    gray = cv.cvtColor(img_blur, cv.COLOR_RGB2GRAY)
    gray = img_blur[:,:,0]
    ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=9)

    # Sure background area
    sure_bg = cv.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    thr = 0.1 * dist_transform.max()
    thr = 18
    ret, sure_fg = cv.threshold(dist_transform, thr, 255, 0)
    
    tools.show_image_chain([sure_fg, sure_bg], titles=["Sure FG", "Sure BG"])
    tools.show_image_chain([opening, dist_transform], titles=["Opening", "Distance transform"])

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)
    
    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)
    
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    
    markers = cv.watershed(img,markers)
    img[markers == -1] = [255,0,0]
    
    return markers, img
    
    
img = img1.copy()
markers, result = segment_red_blood_cells_watershed(img=img)
tools.show_image_chain([markers, result], 
                       titles=["Markers", "Segmented"])

---

## **AI driven segmentation**
Nowadays, deep learning is often used for image segmentation. The U-Net
architecture is a popular choice for this task. It is a convolutional
neural network that is particularly well suited for segmentation tasks.
The nnU-Net is a more advanced version of the U-Net that has been optimized
for medical image segmentation, and comes with self-adapting preprocessing
and postprocessing steps, where the network learns the optimal parameters
for the task at hand. It is available as a Python package and can be installed
via pip.

Although machine learning / artificial intelligence is not the focus of this
course, we can use such models to perform image segmentation. In contrast to 
the methods described above, deep learning models can learn the relevant 
features from the data and can generalize to unseen data. However, these methods
require large amounts of labeled data for training, and are computationally
more expensive. They are also often considered as "black boxes", as it is
difficult to understand why the model makes a certain decision.

```python
######################
###    EXCERISE    ###
######################
```

Visit the following resources and examine if they could be useful for your own segmentation project.
- Segment anything by Meta AI. [Demo](https://segment-anything.com/demo), [Paper](https://arxiv.org/abs/2304.02643), [Code](https://github.com/facebookresearch/segment-anything) 
- Huggingface: Collection of public, pre-trained models. [Link](https://huggingface.co/spaces).
  - Some models come with a demo interface. 
  - For instance, the background removal [RemBG](https://huggingface.co/spaces/KenjieDec/RemBG) tool is found there.
  - Another popular segmentation tool is [Demo](https://huggingface.co/spaces/fcakyon/yolov8-segmentation), [Code](https://huggingface.co/spaces/fcakyon/yolov8-segmentation)
  - To search the entire database for models: [Link](https://huggingface.co/models)
- Total segmentator for anatomical CT data. [Demo](https://totalsegmentator.com/), [Paper](https://arxiv.org/abs/2208.05868), [Code](https://github.com/wasserth/TotalSegmentator)


We have seen now a couple of approaches to segment images. How you can make best
use of them depends on the specific problem and your engineering skills. 😊

In [ ]:
######################
###    SOLUTION    ###
######################

# Let's use the Segment Anything model from Meta.
# The following lines may take a while to execute.
try:
    from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
except ImportError:
    print("Installing the model...")
    !pip install -q git+https://github.com/facebookresearch/segment-anything.git
    !pip install -q opencv-python pycocotools matplotlib onnxruntime onnx
    !pip install -q torch torchvision

# Download the model (if not available yet)
path_to_checkpoint="./sam_vit_h_4b8939.pth"
url_checkpoint="https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
if not Path(path_to_checkpoint).exists():
    print("Downloading model... This may take a while!")
    !wget -O {path_to_checkpoint} {url_checkpoint}

# Load model
print("Loading model...")
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint=path_to_checkpoint)
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
def blend(img, overlay):
    """Blend an image with an overlay."""
    alpha = overlay[:,:,3]
    img = img.astype(np.float32)
    result = ((1 - alpha[:, :, None]) * img + 
              alpha[:, :, None] * overlay[:, :, :3] * 255)
    result = result / result.max()
    return result

In [ ]:
def segment_blood_cells_sam(img, masks):
    result = img.copy()
    overlay_color = [255, 255, 0]
    alpha = 0.2
    clean_masks = True

    # Sort masks by area
    masks = sorted(masks, key=(lambda x: x["area"]), reverse=True)

    if clean_masks:
        # Filter masks that are fully mask
        to_remove = []
        for i, m1 in enumerate(masks):
            m1 = m1["segmentation"]
            for j in range(i+1, len(masks)):
                m2 = masks[j]["segmentation"]
                if i in to_remove or j in to_remove:
                    continue
                if (m1.sum()) ==( (m1 | m2).sum()):
                    to_remove.append(j)
        masks = [m for i, m in enumerate(masks) if i not in to_remove]
        
    # Check the type of cell, using the following heuristic:
    # If the mask is mostly red, it is a red blood cell, if
    # it is mostly blue, it is a white blood cell
    for m in masks:
        mask = m["segmentation"]
        r = img[:,:,0][mask].mean()
        b = img[:,:,2][mask].mean()
        m["type"] = "rbc" if r > b else "wbc"
        
    # Visualize the masks
    result = np.ones((img.shape[0], img.shape[1], 4))
    result[:,:,3] = 0
    for m in masks:
        cell = m["type"]
        m = m["segmentation"]
        contours, hierarchy = cv.findContours(m.astype(np.uint8)*255, 
                                                cv.RETR_TREE, 
                                                cv.CHAIN_APPROX_SIMPLE)
        # Random numbers for the color
        rr, rb, rg = np.random.random(3)*0.5
        overlay_color = ([255/255, rb, rg, alpha] if (cell == "rbc") 
                         else [rr, rg, 255/255, alpha])
        result[m] = overlay_color
        overlay_color[3] = 1
        cv.drawContours(result, contours, -1, overlay_color, 2)
        
    result = blend(img, result)
    tools.show_image_chain([img, result], titles=["Input", "Output"])

In [ ]:
# Choose image here:
img = img1

# Generate the masks (this may take a while, about 30s)
# Keeping this call outside the segment* function because 
# this step takes a while.
print("Generating mask... This may take a while!")
masks = mask_generator.generate(img)

# Visualize the masks
segment_blood_cells_sam(img, masks)

In [ ]:
# Create a binary mask for the red blood cells and save it to a file
mask = sum([m["segmentation"] for m in masks if m.get("type") == "rbc"])
# Apply some morphological opening to clean the mask
mask = cv.morphologyEx(mask.astype(np.uint8), cv.MORPH_OPEN, 
                       np.ones((3, 3), np.uint8), iterations=1)
tools.show_image(mask.astype(np.uint8)*255, title=None, suppress_info=True)
cv.imwrite("mask_rbc.png", mask.astype(np.uint8)*255)

The result looks very good now. The model is able to segment the 
different cells very well. This is quite impressive, considering that
the model was trained on a general purpose dataset and not on medical
images. The model is able to generalize well to new domains.

In [ ]:
# Repeat for img2
img = img2
masks = mask_generator.generate(img)
segment_blood_cells_sam(img, masks)

# Repeat for img3
img = img3
masks = mask_generator.generate(img)
segment_blood_cells_sam(img, masks)